In [1]:
## importing all libraries and dependencies
import pandas as pd
import numpy as np
import os
from os import listdir
import glob
from os.path import isfile, join
import shutil
import PIL
from PIL import Image
import random
import math
from heapq import merge
from random import *
import scipy as sp
from scipy import stats
import seaborn as sns
from matplotlib import pyplot as plt

In [731]:
#for the group df that'll be made
participants = []
all_safe1 = []
all_threat1 = []
all_safe2 = []
all_threat2 = []
all_safe3 = []
all_threat3 = []
all_safe4 = []
all_threat4 = []
all_safe5 = []
all_threat5 = []

## SCR data

In [2]:
dirpath = '/Users/ayesh/Desktop/Gradstuff/Temp_mem2022/SCR_cut3/'
dir_ext = [f for f in listdir(dirpath)]
pt_dir_lst = []
for i in dir_ext:
    pt_dir = dirpath + i
    pt_dir_lst.append(pt_dir)

In [688]:
pt_dir_lst
pt_num = '74'
run_num = '5'
pt = 'emt_'+pt_num+'(txt)'

In [689]:
for one in pt_dir_lst:
    pt = 'emt_'+pt_num+'(txt)'
    if pt == one[-11:]:   
        pt_files = [f for f in listdir(one)]
        block = pt_files[int(run_num)-1]
        # read text file into pandas DataFrame
        a = one +'/'+ block
        df = pd.read_csv(a, sep="\t", header=None, skiprows = 11)
        df.drop(columns=[df.columns[-1]], inplace=True)
        df.columns = ["time", "GSR","shock","stims"]
        print(df.head())

In [690]:
##finding the starting index to slice according to stims and context 
## sample rate is 5msec so Stim readings are 700 while the iti is 200
tmp = df.stims.idxmax()
tmp2 = df[(tmp-700):] #since the acq is actually marking + as the stim, we have to get the first 700 readings of the first stim
#tmp3 = tmp2[tmp2.stims == 0].index[0]
#diff = tmp3 - tmp

##testing the indices are right
print(tmp)
print(tmp2)# df starting with stim
#print(tmp3) # index of first 0 after stim
#print(diff) # stim interval = 200
#tmp4 = df[df.index == tmp3]
#tmp5 = df[tmp3:]
#print(tmp5)
#tmp6 = tmp5.stims.idxmax() # first occurence of stim
#print(tmp6)
#print(df[df.index == tmp6])
#diff2 = tmp6 - tmp3 
#print(diff2) # iti (actually stim) interval = 700


#startpt = tmp - 10000 ## arbitrary buffer period before stims begin; maybe dont need it
#startpt = tmp # 1st stim onset

#1. add stim and iti time then slice the df every n rows of that number
trial_time = 900
cntxt_time = trial_time * 6
cntxts = range(19)
trls_rec = list(range(5400))* len(cntxts) # trials_rec for cutting out first two trials off the context
cntxt_df = pd.DataFrame({'contexts': cntxts})
trls_rdf = pd.DataFrame({'trls_rec': trls_rec})
cntxt_df
new_cntxtdf = pd.DataFrame(np.repeat(cntxt_df.values, cntxt_time, axis=0))
#new_trldf = pd.DataFrame(np.repeat(trls_rdf.values, 19, axis=0))
new_recdf = tmp2.reset_index()
new_recdf

###put new columns with context1,2,3 so on and trial recordings numbered through 5400 for each context
new_df = pd.concat([new_recdf, new_cntxtdf], axis = 1)
new_df.rename(columns = {0:'contexts'}, inplace = True)
new_df2 = pd.concat([new_df, trls_rdf], axis = 1)
new_df2.rename(columns = {0:'trls_rec'}, inplace = True)

4705
          time      GSR  shock  stims
4005    0.3337  0.06561      0      0
4006    0.3338  0.06714      0      0
4007    0.3339  0.07324      0      0
4008    0.3340  0.07477      0      0
4009    0.3341  0.07477      0      0
...        ...      ...    ...    ...
107625  8.9690 -0.06409      0      0
107626  8.9690 -0.06561      0      0
107627  8.9690 -0.06409      0      0
107628  8.9690 -0.06714      0      0
107629  8.9690 -0.06714      0      0

[103625 rows x 4 columns]


In [691]:
#new_df2.to_csv('tst_newdf2.csv')

In [692]:
cut_df = new_df2[new_df2.trls_rec > 2699]
cut_df#.to_csv('tst_cutdf.csv')

,index,time,GSR,shock,stims,contexts,trls_rec
2700,6705,0.5587,-0.03052,0,0,0.0,2700.0
2701,6706,0.5588,-0.02899,0,0,0.0,2701.0
2702,6707,0.5589,-0.02899,0,0,0.0,2702.0
2703,6708,0.5590,-0.03357,0,0,0.0,2703.0
2704,6709,0.5591,-0.03204,0,0,0.0,2704.0
...,...,...,...,...,...,...,...
102595,106600,8.8830,-0.03204,0,5,18.0,5395.0
102596,106601,8.8830,-0.02594,0,5,18.0,5396.0
102597,106602,8.8830,-0.03052,0,5,18.0,5397.0
102598,106603,8.8840,-0.02747,0,5,18.0,5398.0


In [693]:
#2.put this line of code categorizing by context #agg = tmp2.groupby('stims')['GSR'].agg(['count', 'mean']).reset_index() 
agg = cut_df.groupby('contexts')['GSR'].agg(['count', 'mean']).round(4).reset_index() 
agg.columns = [*agg.columns[:-1], 'mean_scr']

In [694]:
agg#.to_csv('tst_agg.csv')

,contexts,count,mean_scr
0,0.0,2700,-0.0478
1,1.0,2700,0.0301
2,2.0,2700,-0.0477
3,3.0,2700,-0.0215
4,4.0,2700,-0.0416
5,5.0,2700,0.0644
6,6.0,2700,-0.1093
7,7.0,2700,0.0064
8,8.0,2700,-0.0264
9,9.0,2700,-0.0354


## encoding context

In [695]:
encod_dirs = '/Users/ayesh/Desktop/Gradstuff/Temp_mem2022/summary_data/new_batch/combined/'
encod_ext = [f for f in listdir(encod_dirs)]
for i in encod_ext:
    if i[-2:]== pt_num:
        pt_encod = encod_dirs + i

In [696]:
pt_encod

'/Users/ayesh/Desktop/Gradstuff/Temp_mem2022/summary_data/new_batch/combined/emt_74'

In [697]:
encoding_files = []
pt_encod_files = [f for f in listdir(pt_encod)]
        
for a in pt_encod_files:
    if a[-4:] == '.csv' and 'encoding' in a:
        encoding_files.append(pt_encod + '/' + a)

In [698]:
pt_encod

'/Users/ayesh/Desktop/Gradstuff/Temp_mem2022/summary_data/new_batch/combined/emt_74'

In [699]:
for file in encoding_files:
    file_num = file [-29]
    if (file_num == run_num):
        enc_file = pd.read_csv(file)

In [700]:
enc_file

,context_num,context_type,context_shock,cntxt_trialnum,shock_object,us,shock_time,contexts_pics,objects_pics,contexts.thisRepN,...,fix_3.stopped,inst_test1_5.started,key_resp_4.started,participant,session,date,expName,psychopyVersion,frameRate,Unnamed: 33
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,2023-10-13_16h14.47.009,encoding5,2022.2.1,59.41107,NaN
1,0.0,SAFE,None,1.0,None,0.0,None,Scenes_selected/scene_010.jpg,new_obs/1_190.jpg,0.0,...,NaN,NaN,NaN,NaN,1.0,2023-10-13_16h14.47.009,encoding5,2022.2.1,59.41107,NaN
2,0.0,SAFE,None,2.0,None,0.0,None,Scenes_selected/scene_010.jpg,new_obs/1_108.jpg,0.0,...,NaN,NaN,NaN,NaN,1.0,2023-10-13_16h14.47.009,encoding5,2022.2.1,59.41107,NaN
3,0.0,SAFE,None,3.0,None,0.0,None,Scenes_selected/scene_010.jpg,new_obs/object_371.jpg,0.0,...,50.150159,NaN,NaN,NaN,1.0,2023-10-13_16h14.47.009,encoding5,2022.2.1,59.41107,NaN
4,0.0,SAFE,None,4.0,None,0.0,None,Scenes_selected/scene_010.jpg,new_obs/1_265.jpg,0.0,...,NaN,NaN,NaN,NaN,1.0,2023-10-13_16h14.47.009,encoding5,2022.2.1,59.41107,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,16.0,SAFE,None,3.0,None,0.0,None,Scenes_selected/scene_080.jpeg,new_obs/1_434.jpg,0.0,...,NaN,NaN,NaN,NaN,1.0,2023-10-13_16h14.47.009,encoding5,2022.2.1,59.41107,NaN
112,16.0,SAFE,None,4.0,None,0.0,None,Scenes_selected/scene_080.jpeg,new_obs/1_207.jpg,0.0,...,NaN,NaN,NaN,NaN,1.0,2023-10-13_16h14.47.009,encoding5,2022.2.1,59.41107,NaN
113,16.0,SAFE,None,5.0,None,0.0,None,Scenes_selected/scene_080.jpeg,new_obs/1_39.jpg,0.0,...,NaN,NaN,NaN,NaN,1.0,2023-10-13_16h14.47.009,encoding5,2022.2.1,59.41107,NaN
114,16.0,SAFE,None,6.0,None,0.0,None,Scenes_selected/scene_080.jpeg,new_obs/1_216.jpg,0.0,...,NaN,NaN,NaN,NaN,1.0,2023-10-13_16h14.47.009,encoding5,2022.2.1,59.41107,NaN


In [701]:
#3 collapse the encoding contexts to just say the context num and type
enc_df = enc_file[["context_num", "context_type", "context_shock", "cntxt_trialnum"]]
nona= enc_df.dropna()
cln = nona.iloc[::6, :]
butt = cln.reset_index()

In [702]:
butt.to_csv('tst_butt.csv')

In [703]:
#4 match the agg version with the collapsed encoding contexts
mrgd = pd.concat([agg, butt], axis = 1)
mrgd['run_num'] = run_num

# throw out shock context entirely
mrgd_clnd = mrgd[mrgd.context_shock != 'shock-cntxt']

# zscore the scl values
zscores = []
meu = round(np.mean(mrgd_clnd.mean_scr), 4)
std_cntxt = round(np.std(mrgd_clnd.mean_scr), 4)
for index, row in mrgd_clnd.iterrows():
    obs = mrgd_clnd.mean_scr[index]
    z_cntxt = (obs - meu)/std_cntxt
    zscores.append (z_cntxt)
mrgd_clnd['zcores'] = zscores

#output the csv in the scr folder
output_dir = dirpath + pt
mrgd_clnd.to_csv(output_dir+"/run%s.csv" %run_num, index = False, header = True)
print (meu, std_cntxt)

0.0641 0.5036


<ipython-input-703-3fe3aee55532>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mrgd_clnd['zcores'] = zscores


In [704]:
mrgd_clnd

,contexts,count,mean_scr,index,context_num,context_type,context_shock,cntxt_trialnum,run_num,zcores
0,0.0,2700,-0.0478,1,0.0,SAFE,None,1.0,5,-0.222200
1,1.0,2700,0.0301,7,1.0,SAFE,None,1.0,5,-0.067514
2,2.0,2700,-0.0477,13,2.0,SAFE,None,1.0,5,-0.222002
3,3.0,2700,-0.0215,19,3.0,THREAT,None,1.0,5,-0.169976
4,4.0,2700,-0.0416,25,4.0,SAFE,prior,1.0,5,-0.209889
6,6.0,2700,-0.1093,37,4.0,SAFE,post,1.0,5,-0.344321
7,7.0,2700,0.0064,43,5.0,THREAT,None,1.0,5,-0.114575
8,8.0,2700,-0.0264,49,6.0,THREAT,None,1.0,5,-0.179706
9,9.0,2700,-0.0354,55,7.0,THREAT,None,1.0,5,-0.197577
10,10.0,2700,-0.0107,61,8.0,THREAT,None,1.0,5,-0.148531


### data cleaning/ summarizing

In [726]:
#5 summarize the pt dataframes for ttesting in R

#6 find if threat and safe contexts differ
pt_num = '74'
pt = 'emt_'+pt_num+'(txt)'

In [727]:
#only run after youve generated all run files for the participants
output_dir = dirpath + pt
runfiles = [f for f in listdir(output_dir)]
csvs = []
for a in runfiles:
    if a[-4:] == '.csv':
        one_run = output_dir +'/' + a
        csvs.append(pd.read_csv(one_run))
all_csvs = pd.concat(csvs, ignore_index = True)
all_csvs.to_csv(output_dir+"/summary%s.csv" %pt_num, index = False, header = True)

In [728]:
summary_df = pd.read_csv(output_dir+"/summary%s.csv" %pt_num)
melted_df = pd.melt(summary_df, id_vars=['run_num'], value_vars=['zcores'], var_name='context_cond', value_name='mean')

## reading the summary csv

In [746]:
## reading the summary csv
pt_num = '74'
pt = 'emt_'+pt_num+'(txt)'
rd = pd.read_csv(dirpath+pt+"/summary%s.csv" %pt_num)#'/Users/ayesh/Desktop/Gradstuff/Temp_mem2022/SCR/emt_66(txt)/summary66.csv')
participants.append(pt_num)

In [747]:
#for zscore the overall, the safe and a threat; overall mean and std deviation first, then the zscore (subtract the context mean from the overall mean)
# for each pt, get safety and threat value per run, append for a group dfs
safe1 = rd.loc[(rd['run_num'] == 1) & (rd['context_type'] == 'SAFE')]
threat1 = rd.loc[(rd['run_num'] == 1) & (rd['context_type'] == 'THREAT')]

safe2 = rd.loc[(rd['run_num'] == 2) & (rd['context_type'] == 'SAFE')]
threat2 = rd.loc[(rd['run_num'] == 2) & (rd['context_type'] == 'THREAT')]

safe3 = rd.loc[(rd['run_num'] == 3) & (rd['context_type'] == 'SAFE')]
threat3 = rd.loc[(rd['run_num'] == 3) & (rd['context_type'] == 'THREAT')]

safe4 = rd.loc[(rd['run_num'] == 4) & (rd['context_type'] == 'SAFE')]
threat4 = rd.loc[(rd['run_num'] == 4) & (rd['context_type'] == 'THREAT')]

safe5 = rd.loc[(rd['run_num'] == 5) & (rd['context_type'] == 'SAFE')]
threat5 = rd.loc[(rd['run_num'] == 5) & (rd['context_type'] == 'THREAT')]

safe1_mn = np.mean(safe1.zcores)
threat1_mn = np.mean(threat1.zcores)
all_safe1.append(safe1_mn)
all_threat1.append(threat1_mn)

safe2_mn = np.mean(safe2.zcores)
threat2_mn = np.mean(threat2.zcores)
all_safe2.append(safe2_mn)
all_threat2.append(threat2_mn)


safe3_mn = np.mean(safe3.zcores)
threat3_mn = np.mean(threat3.zcores)
all_safe3.append(safe3_mn)
all_threat3.append(threat3_mn)


safe4_mn = np.mean(safe4.zcores)
threat4_mn = np.mean(threat4.zcores)
all_safe4.append(safe4_mn)
all_threat4.append(threat4_mn)


safe5_mn = np.mean(safe5.zcores)
threat5_mn = np.mean(threat5.zcores)
all_safe5.append(safe5_mn)
all_threat5.append(threat5_mn)


## group compiled csv reading

In [2163]:
#participants = participants + ['53']
#participants = ['34', '35', '52', '53', '58', '59', '59', '53','35','39','45']

In [748]:
#read the group csv
# participant 36 was added twice so ran this line but commented for future:
#participants = list(np.unique(participants))
grp_df = pd.DataFrame({'participant': participants,
                       'safe_run1':all_safe1,
                       'threat_run1':all_threat1,
                      'safe_run2': all_safe2,
                      'threat_run2': all_threat2,
                      'safe_run3':all_safe3,
                       'threat_run3':all_threat3,
                      'safe_run4': all_safe4,
                      'threat_run4': all_threat4,
                      'safe_run5':all_safe5,
                    'threat_run5':all_threat5})

In [749]:
grp_df

,participant,safe_run1,threat_run1,safe_run2,threat_run2,safe_run3,threat_run3,safe_run4,threat_run4,safe_run5,threat_run5
0,67,-0.424088,0.429177,-0.046218,0.042951,-0.333828,0.333828,0.664758,-0.532061,-0.261980,0.261936
1,68,0.227370,-0.227459,0.261385,-0.261429,-0.253523,0.253412,-0.201789,0.161551,0.181692,-0.227164
2,69,0.227370,-0.227459,0.261385,-0.261429,-0.253523,0.253412,-0.201789,0.161551,-0.201789,0.161551
3,70,0.227370,-0.227459,0.261385,-0.261429,-0.253523,0.253412,-0.201789,0.161551,0.181692,-0.227164
4,71,0.227370,-0.227459,0.261385,-0.261429,-0.253523,0.253412,-0.201789,0.161551,0.181692,-0.227164
5,72,0.227370,-0.227459,0.261385,-0.261429,-0.253523,0.253412,-0.201789,0.161551,0.181692,-0.227164
6,73,0.227370,-0.227459,0.261385,-0.261429,-0.253523,0.253412,-0.201789,0.161551,0.181692,-0.227164
7,74,0.227370,-0.227459,0.261385,-0.261429,-0.253523,0.253412,-0.201789,0.161551,0.181692,-0.227164


In [2167]:
grp_df.to_csv('/Users/ayesh/Desktop/Gradstuff/Temp_mem2022/SCR_cut3/grp_scr_newbatch3.csv')

10